# Ufo

## Dependencias

In [1]:
# data wrangling
import pandas as pd
import numpy as np
import unicodedata
import re
import datetime

# Visualizacion
import seaborn as sb
import matplotlib.pyplot as plt
from plotly.offline import plot,iplot
import plotly.express as px
import plotly.graph_objects as go

# Display options
import warnings 
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 35)

In [2]:
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"}) 
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

In [3]:
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    text=str(text)
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().lstrip().split())
    return cleaned_text if cleaned_text!="nan" else np.nan

## Lectura de datos

In [4]:
df=pd.read_csv('../data/raw/scrubbed.csv',error_bad_lines=False,low_memory=False)
df.head(3)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667


In [5]:
df.shape

(80332, 11)

## Calidad de datos

### Completitud

In [6]:
completitud(df)

,columna,total,completitud
0,country,9670,87.962456
1,state,5797,92.783698
2,shape,1932,97.594981
3,comments,15,99.981327
4,datetime,0,100.000000
5,city,0,100.000000
6,duration (seconds),0,100.000000
7,duration (hours/min),0,100.000000
8,date posted,0,100.000000
9,latitude,0,100.000000


In [7]:
# Debido a que ninguna variable cae por debajo del 80% de completitud, podemos continuar trabajando con el 
# dataset completo

### Duplicados

In [8]:
n_duplicados=df.duplicated().sum()#Total de registros duplicados
print(f"Número de duplicados general : { n_duplicados}")

Número de duplicados general : 0


### Orden y consistencia

In [9]:
df.head(1)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111


#### Renombramos las cols

In [10]:
# en primer lugar modificamos los titulos de las variables a que esten en minusculas
df = df.rename(columns=str.lower)
df = df.rename(columns=lambda x: x.strip().replace(' ', ''))
df.head(1)

,datetime,city,state,country,shape,duration(seconds),duration(hours/min),comments,dateposted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111


In [11]:
df.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration(seconds)',
       'duration(hours/min)', 'comments', 'dateposted', 'latitude',
       'longitude'],
      dtype='object')

In [12]:
# ahora veamos los valores que contiene cada variable
for col in df.columns:
    print(col)
    print('------------------------------------')
    display(df[col].value_counts())

datetime
------------------------------------


7/4/2010 22:00      36
7/4/2012 22:00      31
11/16/1999 19:00    27
9/19/2009 20:00     26
7/4/2011 22:00      25
                    ..
2/21/2005 22:00      1
2/21/2006 20:05      1
2/21/2006 20:52      1
2/21/2006 21:00      1
9/9/2013 23:00       1
Name: datetime, Length: 69586, dtype: int64

city
------------------------------------


seattle                                  525
phoenix                                  454
portland                                 374
las vegas                                368
los angeles                              353
                                        ... 
playa del cura (grand canaria island)      1
livermore/pleasonton (area)                1
hamburg (germany)                          1
cole camp                                  1
hamstead (hollyridge)                      1
Name: city, Length: 19900, dtype: int64

state
------------------------------------


...

12/12/2009    1510
10/30/2006    1408
11/21/2010    1237
10/31/2008    1164
3/19/2009     1098
              ... 
3/1/2004         4
1/20/2003        2
5/23/1999        2
1/10/2003        1
3/25/2002        1
Name: dateposted, Length: 317, dtype: int64

latitude
------------------------------------


47.6063889    581
40.7141667    529
33.4483333    475
34.0522222    392
36.1750000    388
             ... 
19.449345       1
41.3280556      1
37.2908333      1
53.717856       1
50.465843       1
Name: latitude, Length: 18445, dtype: int64

longitude
------------------------------------


-122.330833    581
-74.006389     529
-112.073333    475
-118.242778    392
-115.136389    388
              ... 
-121.711667      1
-0.383333        1
-71.909167       1
-77.395247       1
 22.891814       1
Name: longitude, Length: 19455, dtype: int64

In [13]:
df.head(1)

,datetime,city,state,country,shape,duration(seconds),duration(hours/min),comments,dateposted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111


In [14]:
# "c_"  - Variables numericas : Discretas y continuas
# "v_"  - Variables categoricas
# "d_"  - Variables tipo fecha
# "t_"  - Variables de texto : comentarios, descripciones, url
c_feats=["duration(seconds)",'latitude','longitude']
v_feats=["state",'country',"shape"]
t_feats=['city','duration(hours/min)','comments']
d_feats=["datetime",'dateposted']

In [15]:
c_feats_new=["c_"+x for x in c_feats]
v_feats_new=["v_"+x for x in v_feats]
t_feats_new=["t_"+x for x in t_feats]
d_feats_new=["d_"+x for x in d_feats]

df.rename(columns=dict(zip(d_feats,d_feats_new)),inplace=True) # EL TRUE HACE QUE EL DATAFRAME ORIGINAL SE RENOMBRE 
df.rename(columns=dict(zip(t_feats,t_feats_new)),inplace=True)
df.rename(columns=dict(zip(v_feats,v_feats_new)),inplace=True)
df.rename(columns=dict(zip(c_feats,c_feats_new)),inplace=True)
df.head(2)

,d_datetime,t_city,v_state,v_country,v_shape,c_duration(seconds),t_duration(hours/min),t_comments,d_dateposted,c_latitude,c_longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082


#### data type

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   d_datetime             80332 non-null  object 
 1   t_city                 80332 non-null  object 
 2   v_state                74535 non-null  object 
 3   v_country              70662 non-null  object 
 4   v_shape                78400 non-null  object 
 5   c_duration(seconds)    80332 non-null  object 
 6   t_duration(hours/min)  80332 non-null  object 
 7   t_comments             80317 non-null  object 
 8   d_dateposted           80332 non-null  object 
 9   c_latitude             80332 non-null  object 
 10  c_longitude            80332 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.7+ MB


In [17]:
# extraememos unicamente los caracteres que hacen referencia a la fecha por dias
#df['Fecha'] = df['Fecha'].str.extract(r'(\d{1,2}/\d{1,2}/\d{4})')
df['d_datetime'] = df['d_datetime'].str.extract(r'(\d{1,2}/\d{1,2}/\d{4})')
df['d_dateposted'] = df['d_dateposted'].str.extract(r'(\d{1,2}/\d{1,2}/\d{4})')
df

,d_datetime,t_city,v_state,v_country,v_shape,c_duration(seconds),t_duration(hours/min),t_comments,d_dateposted,c_latitude,c_longitude
0,10/10/1949,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
...,...,...,...,...,...,...,...,...,...,...,...
80327,9/9/2013,nashville,tn,us,light,600,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.1658333,-86.784444
80328,9/9/2013,boise,id,us,circle,1200,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.6136111,-116.202500
80329,9/9/2013,napa,ca,us,other,1200,hour,Napa UFO&#44,9/30/2013,38.2972222,-122.284444
80330,9/9/2013,vienna,va,us,circle,5,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.9011111,-77.265556


In [18]:
# ahora si transformamos las variables de fecha al tipo de dato correcto:
df['d_datetime']=pd.to_datetime(df['d_datetime'])
df['d_dateposted']=pd.to_datetime(df['d_dateposted'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   d_datetime             80332 non-null  datetime64[ns]
 1   t_city                 80332 non-null  object        
 2   v_state                74535 non-null  object        
 3   v_country              70662 non-null  object        
 4   v_shape                78400 non-null  object        
 5   c_duration(seconds)    80332 non-null  object        
 6   t_duration(hours/min)  80332 non-null  object        
 7   t_comments             80317 non-null  object        
 8   d_dateposted           80332 non-null  datetime64[ns]
 9   c_latitude             80332 non-null  object        
 10  c_longitude            80332 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(8)
memory usage: 6.7+ MB


In [19]:
# ahora limpiemos aquellas variables que contienen texto en sus registros:
# usamos la funcion previamente definida, de nombre: "clean_text"
for col in df.columns:
    if col.startswith('v_'):
        df[col]=df[col].astype(str)
        df[col] = df[col].map(lambda x:clean_text(x.lower()))

In [20]:
df.head(2)

,d_datetime,t_city,v_state,v_country,v_shape,c_duration(seconds),t_duration(hours/min),t_comments,d_dateposted,c_latitude,c_longitude
0,1949-10-10,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,2004-04-27,29.8830556,-97.941111
1,1949-10-10,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.38421,-98.581082


In [21]:
for col in df.columns:
    if col.startswith('t_'):
        df[col]=df[col].astype(str)
        df[col] = df[col].map(lambda x:clean_text(x.lower()))

In [22]:
df.head(2)

,d_datetime,t_city,v_state,v_country,v_shape,c_duration(seconds),t_duration(hours/min),t_comments,d_dateposted,c_latitude,c_longitude
0,1949-10-10,san marcos,tx,us,cylinder,2700,45 minutes,this event took place in early fall around 194...,2004-04-27,29.8830556,-97.941111
1,1949-10-10,lackland afb,tx,NaN,light,7200,1 2 hrs,1949 lackland afb 44 tx lights racing across t...,2005-12-16,29.38421,-98.581082


In [25]:
# ahora convertimos a tipo float aquellas variables numericas:
#df['Columna'] = pd.to_numeric(df['Columna'], errors='coerce')
df['c_duration(seconds)'] = pd.to_numeric(df['c_duration(seconds)'], errors='coerce')
df['c_latitude'] = pd.to_numeric(df['c_latitude'], errors='coerce')
df['c_longitude'] = pd.to_numeric(df['c_longitude'], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   d_datetime             80332 non-null  datetime64[ns]
 1   t_city                 80332 non-null  object        
 2   v_state                74535 non-null  object        
 3   v_country              70662 non-null  object        
 4   v_shape                78400 non-null  object        
 5   c_duration(seconds)    80329 non-null  float64       
 6   t_duration(hours/min)  80332 non-null  object        
 7   t_comments             80317 non-null  object        
 8   d_dateposted           80332 non-null  datetime64[ns]
 9   c_latitude             80331 non-null  float64       
 10  c_longitude            80332 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(6)
memory usage: 6.7+ MB


In [26]:
# Ahora ya tenemos limpio este dataframe, el cual usaremos posteriormente para el desarrollo del analisis exploratorio
# Lo exportamos
df.to_csv('../data/processed/df_data_quality', index=False)